# Training on the Flicker HD Faces Dataset

This notebook is used to compute the PSNR of the predicted images. At this point it contains a lot of duplicate code. 

# NOT WORKING UNTIL I REMOVE THIS NOTICE

## Notes

* Notes from Step4 notebook apply. However most of the notes here will be removed as this notebook will simply load the pre-trained model and run the PSNR computations. This *TBD*

* There is a problem with the section that computes the PSNR. It will not work beyond 25% of the images. The reason appears to have been a change with `numpy`. the version that I am using here does not tolerate a mismatch of the sizes of the mask arran and its boolean index. I will likely have to rewrite the codel.  Until I sort this out, this PSNR is only computed for the first 1749 images. The program fails after that. 


## TO DO

* Clean out non-PSNR calculation code

* Fix boolean indexing error in PSNR calculation.

In [1]:
import os
import gc
import datetime
import numpy as np
import pandas as pd
import cv2

from copy import deepcopy
from tqdm import tqdm

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, LambdaCallback
from keras import backend as K
from keras.utils import Sequence
from keras_tqdm import TQDMNotebookCallback

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from IPython.display import clear_output

# Change to the project root path
if os.path.basename(os.getcwd()) != 'PConv-Keras':
    os.chdir('..')
    
from libs.pconv_model import PConvUnet
from libs.util import MaskGenerator

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2
plt.ioff()

# Dataset

In [5]:
VAL_DIR = r"/home/edm/work/mldata/ffhq//"

BATCH_SIZE = 4

# Performance Evaluation
To evaluate the performance of the network, in this notebook I'll try loading the test masks used in the original paper, and see which PSNR scores we get on imagenet.

Note that the 'masks' dataset from the original paper is not in the repo. It has to be downloaded separately. The project page http://masc.cs.gmu.edu/wiki/partialconv gives a ilnk for the test mask datasets:  http://masc.cs.gmu.edu/wiki/uploads/partialconv/mask.zip

**NOTES** 

* Calculating the PSNR is compute intensive. The following took almost 2 hours an used 10 cores.

* There is some code duplication below becau I intend to extract the section below to its own notebook.

In [6]:
# Load weights from previous run
model = PConvUnet()
model.load(
    #r"C:\Users\Mathias Felix Gruber\Documents\GitHub\PConv-Keras\data\logs\imagenet_phase2\weights.26-1.07.h5",
    #"/home/edm/work/dev/PConv-Keras/data/logs/imagenet_phase1_paperMasksweights.04-8.92.h5",
    #r"/home/edm/work/dev/PConv-Keras/data/logs/imagenet_phase1_paperMasksweights.43-4.33.h5",
    r"/home/edm/work/dev/PConv-Keras/data/logs/ffhq_phase2/weights.36-0.56.h5",
    train_bn=False,
    lr=0.00005
)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
## Create Validation Generator

class AugmentingDataGenerator2(ImageDataGenerator):
    def flow_from_directory(self, directory, mask_generator, *args, **kwargs):
        generator = super().flow_from_directory(directory, class_mode=None, *args, **kwargs)        
        seed = None if 'seed' not in kwargs else kwargs['seed']
        while True:
            
            # Get augmentend image samples
            ori = next(generator)

            # Get masks for each image sample            
            mask = np.stack([
                mask_generator.sample(seed)
                for _ in range(ori.shape[0])], axis=0
            )

            # Apply masks to all image sample
            masked = deepcopy(ori)
            masked[mask==0] = 1

            # Yield ([ori, masl],  ori) training batches
            # print(masked.shape, ori.shape)
            gc.collect()
            yield [masked, mask], ori

val_datagen2 = AugmentingDataGenerator2(rescale=1./255)
val_generator2 = val_datagen2.flow_from_directory(
    VAL_DIR, 
    MaskGenerator(512, 512, 3), 
    target_size=(512, 512), 
    batch_size=BATCH_SIZE, 
    classes=['val'], 
    seed=42
)

In [11]:
# Store data
ratios = []
psnrs = []

# Loop through test masks released with paper (from link above)
test_masks = os.listdir(r'data/masks/testing_mask_dataset')
# Limit number ot maks to be equal to size of val dataset (6999 Images)
test_masks = test_masks[:6999]


for filename in tqdm(test_masks):
    
    # Load mask from paper
    filepath = os.path.join(r'data/masks/testing_mask_dataset', filename)
    mask = cv2.imread(filepath) / 255
    ratios.append(mask[:,:,0].sum() / (512 * 512))
    #ratios.append(mask[:,:,0].sum() / (1024* 1024))
    mask = np.array([1-mask for _ in range(BATCH_SIZE)])
    
    # Pick out image from test generator
    test_data = next(val_generator2)
    (_, _), ori = test_data
    
    masked = deepcopy(ori)
  
    #masked[mask==0] = 1  # original code
    #np.where(a < 4, a, -1)  # -1 is broadcast
    masked = np.where(mask==0, 1,0)
    masked = np.where(mask==0, 1, masked)
    
    # Run prediction on image & mask
    pred = model.predict([ori, mask])
    
    # Calculate PSNR
    psnrs.append(-10.0 * np.log10(np.mean(np.square(pred - ori))))
    #pbar.update(1)
    #spbar.set_description("Processing %s" % file, refresh = True)

100%|██████████| 6999/6999 [41:58<00:00,  2.82it/s]


In [9]:
len(psnrs)

6999

In [12]:
# The cell above has a problem and does not finish its computations
# I changed the code below so that it can work with the data that ti
# colected before it . Instead of hardwiring it just measures the size of 
# the array and it uses that to create the pandas dataframe.
#df = pd.DataFrame({'ratios': ratios[:2408], 'psnrs': psnrs})
df = pd.DataFrame({'ratios': ratios[:len(psnrs)], 'psnrs': psnrs})

means, stds = [], []
idx1 = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5]
idx2 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

for mi, ma in zip(idx1, idx2):
    means.append(df[(df.ratios >= mi) & (df.ratios <= ma)].mean())
    stds.append(df[(df.ratios >= mi) & (df.ratios <= ma)].std())
    
pd.DataFrame(means, index=['{}-{}'.format(a, b) for a, b in zip(idx1, idx2)])

,ratios,psnrs
0.01-0.1,0.058166,31.202827
0.1-0.2,0.145236,28.529028
0.2-0.3,0.246844,26.045333
0.3-0.4,0.341071,23.954500
0.4-0.5,0.438136,22.033820
0.5-0.6,0.541535,17.342973


In [10]:
# The cell above has a problem and does not finish its computations
# I changed the code below so that it can work with the data that ti
# colected before it . Instead of hardwiring it just measures the size of 
# the array and it uses that to create the pandas dataframe.
#df = pd.DataFrame({'ratios': ratios[:2408], 'psnrs': psnrs})
df = pd.DataFrame({'ratios': ratios[:len(psnrs)], 'psnrs': psnrs})

means, stds = [], []
idx1 = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5]
idx2 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

for mi, ma in zip(idx1, idx2):
    means.append(df[(df.ratios >= mi) & (df.ratios <= ma)].mean())
    stds.append(df[(df.ratios >= mi) & (df.ratios <= ma)].std())
    
pd.DataFrame(means, index=['{}-{}'.format(a, b) for a, b in zip(idx1, idx2)])

,ratios,psnrs
0.01-0.1,0.058166,31.183182
0.1-0.2,0.145236,28.532214
0.2-0.3,0.246844,26.061248
0.3-0.4,0.341071,23.983323
0.4-0.5,0.438136,21.968208
0.5-0.6,0.541535,17.309921
